In [18]:
import pandas as pd
import numpy as np
pd.options.display.max_rows = 100
pd.options.display.max_columns = 300
from sklearn.model_selection import train_test_split
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from sklearn.metrics import mean_squared_error

import tensorflow as tf
from keras import Sequential
from keras.layers import Dense
from keras.layers import BatchNormalization
# from keras.layers import Dropout
from keras.layers import InputLayer
# from keras.callbacks import EarlyStopping
import keras_tuner as kt

In [19]:
def plot_histograms( df ):
  i=0
  fig = make_subplots( rows=1, cols=df.shape[1],subplot_titles=df.columns.values )
  for feature in df.columns.values:
    fig.add_trace( go.Histogram( x=df[feature], name=feature), row=1,col=i+1 )
    i = i+1

  no_of_features = len(df.columns.values)
  fig.update_layout( bargap=0.2, width= no_of_features*800, height=700 )
  fig.show()
  return

In [20]:
Data_pp_ee = pd.read_csv( '../input/lhc-events-ppee-ppmumu/data_pp_ee-.csv' )
Data_pp_mumu = pd.read_csv( '../input/lhc-events-ppee-ppmumu/data_pp_mumu-.csv' )
Data_pp_z_ee = pd.read_csv( '../input/lhc-events-pp-a-ee-pp-z-ee/data_pp_z_ee-.csv' )

In [21]:
Data_pp_ee

,invariant_mass_electron,px1,py1,ptl1,px2,py2,ptl2,etal1,etal2
0,92.3608,23.58400,41.508000,47.7401,-0.265954,-47.48210,47.4828,0.141171,0.278032
1,90.9088,37.61210,11.656300,39.3769,-33.767800,-15.68980,37.2349,1.493910,0.282925
2,79.7348,-47.86820,-40.110200,62.4515,4.626220,22.87610,23.3392,-1.136180,-0.269716
3,96.3826,45.08040,-3.183530,45.1926,-38.092000,3.51633,38.2539,0.986720,2.100410
4,89.4622,20.61180,-48.042200,52.2771,-3.156180,31.25360,31.4126,-1.598680,-0.651694
...,...,...,...,...,...,...,...,...,...
3778,94.8389,-31.25680,54.505500,62.8318,17.672400,-19.35740,26.2111,-0.901628,0.260424
3779,90.8178,40.18990,0.048142,40.1899,-39.996900,3.61579,40.1600,-1.246720,-0.233096
3780,93.6519,-1.95051,-46.670900,46.7116,3.213300,42.72870,42.8493,0.047952,0.657368
3781,91.1566,-79.74990,51.773000,95.0815,19.345800,-10.17040,21.8563,-0.890719,-0.974265


In [22]:
Data_pp_z_ee

,invariant_mass_electron,px1,py1,ptl1,px2,py2,ptl2,etal1,etal2
0,46.7906,-16.184500,-12.74130,20.5980,11.86830,6.06155,13.3266,-0.140835,-1.905970
1,90.7440,-4.991710,17.73920,18.4281,-2.06191,-18.18030,18.2968,-1.097690,2.016960
2,89.2582,20.693200,-38.08910,43.3473,-7.79677,42.49780,43.2071,-1.794890,-1.209170
3,99.9240,39.643600,39.95740,56.2869,-27.13160,-32.43630,42.2876,1.481710,1.035870
4,85.9435,33.088100,-28.35540,43.5758,-21.72990,37.76000,43.5661,-1.508870,-1.581530
...,...,...,...,...,...,...,...,...,...
4226,93.0547,-0.308304,-45.41870,45.4197,20.68170,27.86280,34.6996,-1.188560,0.093119
4227,93.5257,-46.094800,8.97271,46.9600,45.80830,-4.95354,46.0754,-0.377846,-0.155191
4228,92.7492,53.603500,5.69482,53.9052,-22.87220,12.69490,26.1591,-0.452259,0.989377
4229,94.1083,34.943100,-2.50055,35.0325,-27.53840,5.66127,28.1143,1.272590,-0.653637


In [23]:
Data_pp_mumu.info()
Data_pp_ee.info()
Data_pp_z_ee.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6536 entries, 0 to 6535
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   invariant_mass_moun  6536 non-null   float64
 1   px1                  6536 non-null   float64
 2   py1                  6536 non-null   float64
 3   ptl1                 6536 non-null   float64
 4   px2                  6536 non-null   float64
 5   py2                  6536 non-null   float64
 6   ptl2                 6536 non-null   float64
 7   etal1                6536 non-null   float64
 8   etal2                6536 non-null   float64
dtypes: float64(9)
memory usage: 459.7 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3783 entries, 0 to 3782
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   invariant_mass_electron  3783 non-null   float64
 1   px1                      3783

In [24]:
plot_histograms( Data_pp_ee )

In [25]:
plot_histograms( Data_pp_mumu )

In [26]:
plot_histograms( Data_pp_z_ee )

In [27]:
# Features 
X_1_e = Data_pp_ee[['px1', 'py1']]
X_1_mu = Data_pp_mumu[['px1', 'py1']]
X_1_z = Data_pp_z_ee[['px1', 'py1']]
# Target
y_1_e = Data_pp_ee[['ptl1']]
y_1_mu = Data_pp_mumu[['ptl1']]
y_1_z = Data_pp_z_ee[['ptl1']]

### Function for Plotting Loss vs epochs graph, Accuaracy vs epochs graph


In [28]:
def Plot_Loss_Accuracy( history, epochs ):
    loss = history.history['root_mean_squared_error']
    val_loss = history.history['val_root_mean_squared_error']

    epochs = np.linspace(1,epochs,epochs)
    epochs

    fig = go.Figure(data=go.Scatter( x=epochs, y=loss, name='Root_Mean_Squared_Error' ))
    fig.add_trace( go.Scatter( x=epochs, y=val_loss, name='Val_Root_Mean_Squared_Error' ) )
    fig.update_layout( title="Root Mean Squared Error", xaxis_title='epochs' )
    fig.update_layout(legend=dict( orientation="h", yanchor="bottom", y=1.02, xanchor="right",  x=1), width=1000, height=750)    
    fig.show()

## Train Test Split

In [29]:
X_train, X_test, y_train, y_test = train_test_split( X_1_e , y_1_e , test_size=0.20, random_state=10 )
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((3026, 2), (757, 2), (3026, 1), (757, 1))

# One Layer ANN

In [30]:
# Neural network
model = Sequential()
model.add(InputLayer(input_shape=X_train.shape[1]))

model.add(Dense(100, activation='relu'))

model.add(Dense(1, activation='linear'))
model.compile(loss='mean_squared_error' , optimizer='adam', metrics=[tf.keras.metrics.RootMeanSquaredError()])

In [31]:
history = model.fit(X_train,y_train, validation_data=(X_test,y_test), epochs=15 )

Epoch 1/15
95/95 [==============================] - 1s 3ms/step - loss: 1120.0739 - root_mean_squared_error: 33.4675 - val_loss: 195.6624 - val_root_mean_squared_error: 13.9879
Epoch 2/15
95/95 [==============================] - 0s 2ms/step - loss: 45.5122 - root_mean_squared_error: 6.7463 - val_loss: 6.7630 - val_root_mean_squared_error: 2.6006
Epoch 3/15
95/95 [==============================] - 0s 2ms/step - loss: 4.4866 - root_mean_squared_error: 2.1182 - val_loss: 3.7364 - val_root_mean_squared_error: 1.9330
Epoch 4/15
95/95 [==============================] - 0s 2ms/step - loss: 2.9087 - root_mean_squared_error: 1.7055 - val_loss: 2.5950 - val_root_mean_squared_error: 1.6109
Epoch 5/15
95/95 [==============================] - 0s 2ms/step - loss: 1.9759 - root_mean_squared_error: 1.4057 - val_loss: 1.7310 - val_root_mean_squared_error: 1.3157
Epoch 6/15
95/95 [==============================] - 0s 2ms/step - loss: 1.2796 - root_mean_squared_error: 1.1312 - val_loss: 1.1481 - val_root

In [32]:
Plot_Loss_Accuracy( history, epochs=15 )

In [33]:
# Predicting Target 
Data_pp_ee['predicted_ptl1'] = model.predict(X_1_e)

# Plot of Original and Predicted Target Column
columns = ['ptl1','predicted_ptl1']
fig = ff.create_distplot([Data_pp_ee[c] for c in columns], columns, bin_size=2, show_hist=True, show_rug=False)
fig.layout.update({'title': 'p p > e+ e- | Transverse Momentum (ptl1)'})
fig.update_layout(legend=dict( orientation="h", yanchor="bottom", y=1.02, xanchor="right",  x=1), width=1000, height=1000)
fig.show()

RMSE = mean_squared_error( Data_pp_ee['ptl1'], Data_pp_ee['predicted_ptl1'] )
print( " RMSE = ", RMSE )

 RMSE =  0.10091889870491653


Saving Model

In [ ]:
model.save("model.h5")

## for pp_z_e+e-

In [34]:
# Predicting Target 
Data_pp_z_ee['predicted_ptl1'] = model.predict(X_1_z)

# Plot of Original and Predicted Target Column
columns = ['ptl1','predicted_ptl1']
fig = ff.create_distplot([Data_pp_z_ee[c] for c in columns], columns, bin_size=2, show_hist=True, show_rug=False)
fig.layout.update({'title': 'p p > z > e+ e- | Transverse Momentum (ptl1)'})
fig.update_layout(legend=dict( orientation="h", yanchor="bottom", y=1.02, xanchor="right",  x=1), width=1000, height=1000)
fig.show()

RMSE = mean_squared_error( Data_pp_z_ee['ptl1'], Data_pp_z_ee['predicted_ptl1'] )
print( " RMSE = ", RMSE )

 RMSE =  0.1081460839724838


## for pp_mu+mu-

In [35]:
# Predicting Target 
Data_pp_mumu['predicted_ptl1'] = model.predict(X_1_mu)

# Plot of Original and Predicted Target Column
columns = ['ptl1','predicted_ptl1']
fig = ff.create_distplot([Data_pp_mumu[c] for c in columns], columns, bin_size=2, show_hist=True, show_rug=False)
fig.layout.update({'title': 'p p > mu+ mu- | Transverse Momentum (ptl1)'})
fig.update_layout(legend=dict( orientation="h", yanchor="bottom", y=1.02, xanchor="right",  x=1), width=1000, height=1000)
fig.show()

RMSE = mean_squared_error( Data_pp_mumu['ptl1'], Data_pp_mumu['predicted_ptl1'] )
print( " RMSE = ", RMSE )

 RMSE =  0.09770589625995986


***
***

# Two Layers

In [18]:
# Neural network
model = Sequential()
model.add(InputLayer(input_shape=X_train.shape[1]))

model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))

model.add(Dense(1, activation='linear'))
model.compile(loss='mean_squared_error' , optimizer='adam', metrics=[tf.keras.metrics.RootMeanSquaredError()])

In [19]:
history = model.fit(X_train,y_train, validation_data=(X_test,y_test), epochs=25 )

Epoch 1/25
95/95 [==============================] - 1s 4ms/step - loss: 199.7613 - root_mean_squared_error: 14.1337 - val_loss: 0.5591 - val_root_mean_squared_error: 0.7477
Epoch 2/25
95/95 [==============================] - 0s 3ms/step - loss: 0.2132 - root_mean_squared_error: 0.4617 - val_loss: 0.1105 - val_root_mean_squared_error: 0.3325
Epoch 3/25
95/95 [==============================] - 0s 3ms/step - loss: 0.0598 - root_mean_squared_error: 0.2445 - val_loss: 0.0550 - val_root_mean_squared_error: 0.2345
Epoch 4/25
95/95 [==============================] - 0s 3ms/step - loss: 0.0337 - root_mean_squared_error: 0.1836 - val_loss: 0.0370 - val_root_mean_squared_error: 0.1923
Epoch 5/25
95/95 [==============================] - 0s 3ms/step - loss: 0.0239 - root_mean_squared_error: 0.1546 - val_loss: 0.0265 - val_root_mean_squared_error: 0.1628
Epoch 6/25
95/95 [==============================] - 0s 5ms/step - loss: 0.0180 - root_mean_squared_error: 0.1340 - val_loss: 0.0213 - val_root_mean

In [20]:
Plot_Loss_Accuracy( history, epochs=25 )

In [21]:
# Predicting Target 
Data_pp_ee['predicted_ptl1'] = model.predict(X_1_e)

# Plot of Original and Predicted Target Column
columns = ['ptl1','predicted_ptl1']
fig = ff.create_distplot([Data_pp_ee[c] for c in columns], columns, bin_size=2, show_hist=True, show_rug=False)
fig.layout.update({'title': 'p p > e+ e-'})
fig.show()

RMSE = mean_squared_error( Data_pp_ee['ptl1'], Data_pp_ee['predicted_ptl1'] )
print( " RMSE = ", RMSE )

 RMSE =  0.0005561686098234084


In [22]:
# Predicting Target 
Data_pp_mumu['predicted_ptl1'] = model.predict(X_1_mu)

# Plot of Original and Predicted Target Column
columns = ['ptl1','predicted_ptl1']
fig = ff.create_distplot([Data_pp_mumu[c] for c in columns], columns, bin_size=2, show_hist=True, show_rug=False)
fig.layout.update({'title': 'p p > mu+ mu-'})
fig.show()

RMSE = mean_squared_error( Data_pp_mumu['ptl1'], Data_pp_mumu['predicted_ptl1'] )
print( " RMSE = ", RMSE )

 RMSE =  0.0005346413570256387


# Three Layers

In [23]:
# Neural network
model = Sequential()
model.add(InputLayer(input_shape=X_train.shape[1]))

model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))


model.add(Dense(1, activation='linear'))
model.compile(loss='mean_squared_error' , optimizer='adam', metrics=[tf.keras.metrics.RootMeanSquaredError()])

In [24]:
history = model.fit(X_train,y_train, validation_data=(X_test,y_test), epochs=25 )
Plot_Loss_Accuracy( history, epochs=25 )

Epoch 1/25
95/95 [==============================] - 1s 4ms/step - loss: 268.0439 - root_mean_squared_error: 16.3720 - val_loss: 0.2288 - val_root_mean_squared_error: 0.4783
Epoch 2/25
95/95 [==============================] - 0s 3ms/step - loss: 0.0719 - root_mean_squared_error: 0.2681 - val_loss: 0.0328 - val_root_mean_squared_error: 0.1812
Epoch 3/25
95/95 [==============================] - 0s 3ms/step - loss: 0.0314 - root_mean_squared_error: 0.1771 - val_loss: 0.0230 - val_root_mean_squared_error: 0.1518
Epoch 4/25
95/95 [==============================] - 0s 3ms/step - loss: 0.0215 - root_mean_squared_error: 0.1466 - val_loss: 0.0212 - val_root_mean_squared_error: 0.1457
Epoch 5/25
95/95 [==============================] - 0s 3ms/step - loss: 0.0176 - root_mean_squared_error: 0.1328 - val_loss: 0.0126 - val_root_mean_squared_error: 0.1121
Epoch 6/25
95/95 [==============================] - 0s 3ms/step - loss: 0.0131 - root_mean_squared_error: 0.1143 - val_loss: 0.0120 - val_root_mean

In [25]:
# Predicting Target 
Data_pp_ee['predicted_ptl1'] = model.predict(X_1_e)

# Plot of Original and Predicted Target Column
columns = ['ptl1','predicted_ptl1']
fig = ff.create_distplot([Data_pp_ee[c] for c in columns], columns, bin_size=2, show_hist=True, show_rug=False)
fig.layout.update({'title': 'p p > e+ e-'})
fig.show()

RMSE = mean_squared_error( Data_pp_ee['ptl1'], Data_pp_ee['predicted_ptl1'] )
print( " RMSE = ", RMSE )

 RMSE =  0.0008578452064630228


In [26]:
# Predicting Target 
Data_pp_mumu['predicted_ptl1'] = model.predict(X_1_mu)

# Plot of Original and Predicted Target Column
columns = ['ptl1','predicted_ptl1']
fig = ff.create_distplot([Data_pp_mumu[c] for c in columns], columns, bin_size=2, show_hist=True, show_rug=False)
fig.layout.update({'title': 'p p > mu+ mu-'})
fig.show()

RMSE = mean_squared_error( Data_pp_mumu['ptl1'], Data_pp_mumu['predicted_ptl1'] )
print( " RMSE = ", RMSE )

 RMSE =  0.0008461731464027754


# Four Layers

In [27]:
# Neural network
model = Sequential()
model.add(InputLayer(input_shape=X_train.shape[1]))

model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))

model.add(Dense(1, activation='linear'))
model.compile(loss='mean_squared_error' , optimizer='adam', metrics=[tf.keras.metrics.RootMeanSquaredError()])

In [28]:
history = model.fit(X_train,y_train, validation_data=(X_test,y_test), epochs=25 )
Plot_Loss_Accuracy( history, epochs=25 )

Epoch 1/25
95/95 [==============================] - 1s 4ms/step - loss: 152.7915 - root_mean_squared_error: 12.3609 - val_loss: 0.1478 - val_root_mean_squared_error: 0.3845
Epoch 2/25
95/95 [==============================] - 0s 3ms/step - loss: 0.0631 - root_mean_squared_error: 0.2512 - val_loss: 0.0432 - val_root_mean_squared_error: 0.2078
Epoch 3/25
95/95 [==============================] - 0s 3ms/step - loss: 0.0294 - root_mean_squared_error: 0.1715 - val_loss: 0.0379 - val_root_mean_squared_error: 0.1947
Epoch 4/25
95/95 [==============================] - 0s 3ms/step - loss: 0.0185 - root_mean_squared_error: 0.1361 - val_loss: 0.0185 - val_root_mean_squared_error: 0.1359
Epoch 5/25
95/95 [==============================] - 0s 3ms/step - loss: 0.0141 - root_mean_squared_error: 0.1187 - val_loss: 0.0140 - val_root_mean_squared_error: 0.1183
Epoch 6/25
95/95 [==============================] - 0s 3ms/step - loss: 0.0100 - root_mean_squared_error: 0.0998 - val_loss: 0.0120 - val_root_mean

In [29]:
# Predicting Target 
Data_pp_ee['predicted_ptl1'] = model.predict(X_1_e)

# Plot of Original and Predicted Target Column
columns = ['ptl1','predicted_ptl1']
fig = ff.create_distplot([Data_pp_ee[c] for c in columns], columns, bin_size=2, show_hist=True, show_rug=False)
fig.layout.update({'title': 'p p > e+ e-'})
fig.show()

RMSE = mean_squared_error( Data_pp_ee['ptl1'], Data_pp_ee['predicted_ptl1'] )
print( " RMSE = ", RMSE )

 RMSE =  0.0010310242719403957


In [30]:
# Predicting Target 
Data_pp_mumu['predicted_ptl1'] = model.predict(X_1_mu)

# Plot of Original and Predicted Target Column
columns = ['ptl1','predicted_ptl1']
fig = ff.create_distplot([Data_pp_mumu[c] for c in columns], columns, bin_size=2, show_hist=True, show_rug=False)
fig.layout.update({'title': 'p p > mu+ mu-'})
fig.show()

RMSE = mean_squared_error( Data_pp_mumu['ptl1'], Data_pp_mumu['predicted_ptl1'] )
print( " RMSE = ", RMSE )

 RMSE =  0.0009757460018872347


***

# Keras Tuner

In [31]:
def hypertuning_model(hp):
    model = Sequential()
    model.add( InputLayer( input_shape=X_train.shape[1] ))
              
    number_of_layers = hp.Int('No_of_Layers',min_value=2,max_value=6)
    for num_layer in range(1, number_of_layers+1 ):
        model.add( Dense( hp.Int('layer'+str(num_layer), min_value=10, max_value=560, step=50), activation='relu'))
        
    model.add(Dense(1, activation='linear'))    
    model.compile(loss='mean_squared_error' , optimizer='adam', metrics=[tf.keras.metrics.RootMeanSquaredError()])
    optimizer=tf.keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate', values=[1e-1,1e-2, 1e-3, 1e-4])  )
    return model

In [32]:
hyper_tuner_model = kt.Hyperband( hypertuning_model, objective='val_loss',overwrite=True, max_epochs=25,seed=10 )

In [33]:
hyper_tuner_model.search( X_train,y_train, validation_data=(X_test,y_test), epochs=25 )

Trial 30 Complete [00h 00m 08s]
val_loss: 0.00010602112888591364

Best val_loss So Far: 4.3393130908953026e-05
Total elapsed time: 00h 02m 11s


In [34]:
best_hyperparameters = hyper_tuner_model.get_best_hyperparameters(num_trials=1)[0]
best_hyperparameters.values

{'No_of_Layers': 3,
 'layer1': 310,
 'layer2': 260,
 'learning_rate': 0.0001,
 'layer3': 410,
 'layer4': 210,
 'layer5': 60,
 'layer6': 310,
 'tuner/epochs': 25,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 0,
 'tuner/round': 0}

In [35]:
n_best_models = hyper_tuner_model.get_best_models(num_models=1)[0]
n_best_models.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 310)               930       
_________________________________________________________________
dense_1 (Dense)              (None, 260)               80860     
_________________________________________________________________
dense_2 (Dense)              (None, 410)               107010    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 411       
Total params: 189,211
Trainable params: 189,211
Non-trainable params: 0
_________________________________________________________________


# Final Model

In [36]:
final_model = hyper_tuner_model.hypermodel.build( best_hyperparameters )
history = final_model.fit(X_train,y_train, validation_data=(X_test,y_test),epochs=25 )

Epoch 1/25
95/95 [==============================] - 1s 4ms/step - loss: 90.1499 - root_mean_squared_error: 9.4947 - val_loss: 0.0688 - val_root_mean_squared_error: 0.2623
Epoch 2/25
95/95 [==============================] - 0s 4ms/step - loss: 0.0256 - root_mean_squared_error: 0.1600 - val_loss: 0.0114 - val_root_mean_squared_error: 0.1066
Epoch 3/25
95/95 [==============================] - 0s 3ms/step - loss: 0.0085 - root_mean_squared_error: 0.0923 - val_loss: 0.0047 - val_root_mean_squared_error: 0.0683
Epoch 4/25
95/95 [==============================] - 0s 4ms/step - loss: 0.0041 - root_mean_squared_error: 0.0639 - val_loss: 0.0029 - val_root_mean_squared_error: 0.0539
Epoch 5/25
95/95 [==============================] - 0s 5ms/step - loss: 0.0026 - root_mean_squared_error: 0.0507 - val_loss: 0.0019 - val_root_mean_squared_error: 0.0435
Epoch 6/25
95/95 [==============================] - 1s 6ms/step - loss: 0.0018 - root_mean_squared_error: 0.0426 - val_loss: 0.0016 - val_root_mean_s

In [37]:
final_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 310)               930       
_________________________________________________________________
dense_5 (Dense)              (None, 260)               80860     
_________________________________________________________________
dense_6 (Dense)              (None, 410)               107010    
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 411       
Total params: 189,211
Trainable params: 189,211
Non-trainable params: 0
_________________________________________________________________


In [38]:
Plot_Loss_Accuracy( history, epochs=25 )

In [39]:
# Predicting Target 
Data_pp_ee['predicted_ptl1'] = model.predict(X_1_e)

# Plot of Original and Predicted Target Column
columns = ['ptl1','predicted_ptl1']
fig = ff.create_distplot([Data_pp_ee[c] for c in columns], columns, bin_size=2, show_hist=True, show_rug=False)
fig.layout.update({'title': 'p p > e+ e-'})
fig.show()

RMSE = mean_squared_error( Data_pp_ee['ptl1'], Data_pp_ee['predicted_ptl1'] )
print( " RMSE = ", RMSE )

 RMSE =  0.0010310242719403957


In [40]:
# Predicting Target 
Data_pp_mumu['predicted_ptl1'] = model.predict(X_1_mu)

# Plot of Original and Predicted Target Column
columns = ['ptl1','predicted_ptl1']
fig = ff.create_distplot([Data_pp_mumu[c] for c in columns], columns, bin_size=2, show_hist=True, show_rug=False)
fig.layout.update({'title': 'p p > mu+ mu-'})
fig.show()

RMSE = mean_squared_error( Data_pp_mumu['ptl1'], Data_pp_mumu['predicted_ptl1'] )
print( " RMSE = ", RMSE )

 RMSE =  0.0009757460018872347


***

# for pp > z > e+e-

In [41]:
X_train, X_test, y_train, y_test = train_test_split( X_1_e , y_1_e , test_size=0.20, random_state=10 )
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((3026, 2), (757, 2), (3026, 1), (757, 1))

# One Layer ANN

In [42]:
# Neural network
model = Sequential()
model.add(InputLayer(input_shape=X_train.shape[1]))

model.add(Dense(100, activation='relu'))

model.add(Dense(1, activation='linear'))
model.compile(loss='mean_squared_error' , optimizer='adam', metrics=[tf.keras.metrics.RootMeanSquaredError()])

history = model.fit(X_train,y_train, validation_data=(X_test,y_test), epochs=25 )

Plot_Loss_Accuracy( history, epochs=25 )

Epoch 1/25
95/95 [==============================] - 1s 3ms/step - loss: 1098.8379 - root_mean_squared_error: 33.1487 - val_loss: 224.4159 - val_root_mean_squared_error: 14.9805
Epoch 2/25
95/95 [==============================] - 0s 2ms/step - loss: 56.6999 - root_mean_squared_error: 7.5299 - val_loss: 3.6972 - val_root_mean_squared_error: 1.9228
Epoch 3/25
95/95 [==============================] - 0s 2ms/step - loss: 2.4517 - root_mean_squared_error: 1.5658 - val_loss: 1.8499 - val_root_mean_squared_error: 1.3601
Epoch 4/25
95/95 [==============================] - 0s 3ms/step - loss: 1.4657 - root_mean_squared_error: 1.2107 - val_loss: 1.1615 - val_root_mean_squared_error: 1.0777
Epoch 5/25
95/95 [==============================] - 0s 4ms/step - loss: 0.9410 - root_mean_squared_error: 0.9700 - val_loss: 0.7541 - val_root_mean_squared_error: 0.8684
Epoch 6/25
95/95 [==============================] - 0s 3ms/step - loss: 0.5908 - root_mean_squared_error: 0.7686 - val_loss: 0.4723 - val_root

In [43]:
# Predicting Target 
Data_pp_z_ee['predicted_ptl1'] = model.predict(X_1_z)

# Plot of Original and Predicted Target Column
columns = ['ptl1','predicted_ptl1']
fig = ff.create_distplot([Data_pp_z_ee[c] for c in columns], columns, bin_size=2, show_hist=True, show_rug=False)
fig.layout.update({'title': 'p p > z > e+ e-'}, height=700, xaxis_title = "Transverse Momentum (ptl1)", yaxis_title='Probability Density', 
                  font=dict(size=15))
fig.show()

RMSE = mean_squared_error( Data_pp_z_ee['ptl1'], Data_pp_z_ee['predicted_ptl1'] )
print( " RMSE for pp_z_ee data = ", RMSE )

 RMSE for pp_z_ee data =  0.038243532924883895
